In [1]:
import pandas as pd

In [ ]:
df = pd.read_csv("C:\\Users\\ZEESHAN\\Downloads\\home-data-for-ml-course\\train.csv", index_col='Id')


In [ ]:
df_test = pd.read_csv(r"C:\Users\ZEESHAN\Downloads\home-data-for-ml-course\test.csv")

print(df_test.head())

In [ ]:
df.head()

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
print(type(df_test))


In [ ]:
iid=df_test['Id']
X_fin_tech=df_test.drop(columns=['Id'])

In [23]:
categorical_features = ['MSZoning', 'Street', 'Alley', 'LotShape', 'LandContour', 'Utilities', 'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2', 'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st', 'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation', 'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2', 'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual', 'Functional', 'FireplaceQu', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond', 'PavedDrive', 'PoolQC', 'Fence', 'MiscFeature', 'SaleType', 'SaleCondition']
for feature in categorical_features:
    df[feature] = df[feature].astype('category')
    X_fin_tech[feature]=X_fin_tech[feature].astype('category')


In [24]:
y=df['SalePrice']
df.drop(columns=['SalePrice'],inplace=True)
X=df

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.001)

In [29]:
import lightgbm as lgb
train_data = lgb.Dataset(X_train, label=y_train)
test_data = lgb.Dataset(X_test, label=y_test)

In [30]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1460 entries, 1 to 1460
Data columns (total 79 columns):
 #   Column         Non-Null Count  Dtype   
---  ------         --------------  -----   
 0   MSSubClass     1460 non-null   int64   
 1   MSZoning       1460 non-null   category
 2   LotFrontage    1201 non-null   float64 
 3   LotArea        1460 non-null   int64   
 4   Street         1460 non-null   category
 5   Alley          91 non-null     category
 6   LotShape       1460 non-null   category
 7   LandContour    1460 non-null   category
 8   Utilities      1460 non-null   category
 9   LotConfig      1460 non-null   category
 10  LandSlope      1460 non-null   category
 11  Neighborhood   1460 non-null   category
 12  Condition1     1460 non-null   category
 13  Condition2     1460 non-null   category
 14  BldgType       1460 non-null   category
 15  HouseStyle     1460 non-null   category
 16  OverallQual    1460 non-null   int64   
 17  OverallCond    1460 non-null   int64  

In [31]:
params =  {'n_estimators': 693, 'max_depth': 8, 'colsample_bytree': 0.5075413041956253,
           'subsample': 0.83306258044554, 'learning_rate': 0.011639115298956882, 'min_child_samples': 23}
gbm = lgb.train(params,
                train_data,
                num_boost_round=100,
                valid_sets=[train_data, test_data],
                valid_names=['train', 'eval'])

[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=8) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=256) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Warning] Provided parameters constrain tree depth (max_depth=8) without explicitly setting 'num_leaves'. This can lead to underfitting. To resolve this warning, pass 'num_leaves' (<=256) in params. Alternatively, pass (max_depth=-1) and just use 'num_leaves' to constrain model complexity.
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.001276 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 3398
[LightGBM] [Info] Number of data points in the train set: 1458, number of used features: 76
[LightGBM] [Warning] Provided parameters con

In [32]:
fin_pred_lgbm = gbm.predict(X_fin_tech)

In [33]:
fin_pred_lgbm

array([123561.47753256, 157073.53772205, 180258.58128086, ...,
       160571.42495142, 118544.3674252 , 206601.04766727])

In [34]:
result_df = pd.DataFrame({
    'Id': iid,
    'SalePrice': fin_pred_lgbm
})

result_df.to_csv('predictions_lgbm.csv', index=False)